In [ ]:
from aocd import get_data
inp = get_data(day=3, year=2024)

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:

```
xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))
```

Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?

In [ ]:
sample = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

In [ ]:
import re
pat = r'mul\((?P<x>[0-9]{1,3}),(?P<y>[0-9]{1,3})\)'

In [ ]:
?re.match

Signature: re.match(pattern, string, flags=0)
Docstring:
Try to apply the pattern at the start of the string, returning
a Match object, or None if no match was found.
File:      /usr/local/lib/python3.11/re/__init__.py
Type:      function


In [ ]:
m = re.match(pat,'mul(12,123)')
m

<re.Match object; span=(0, 11), match='mul(12,123)'>

In [ ]:
m.group('x')

'12'

In [ ]:
m.group('y')

'123'

In [ ]:
?re.findall

Signature: re.findall(pattern, string, flags=0)
Docstring:
Return a list of all non-overlapping matches in the string.

If one or more capturing groups are present in the pattern, return
a list of groups; this will be a list of tuples if the pattern
has more than one group.

Empty matches are included in the result.
File:      /usr/local/lib/python3.11/re/__init__.py
Type:      function


In [ ]:
?re.finditer

Signature: re.finditer(pattern, string, flags=0)
Docstring:
Return an iterator over all non-overlapping matches in the
string.  For each match, the iterator returns a Match object.

Empty matches are included in the result.
File:      /usr/local/lib/python3.11/re/__init__.py
Type:      function


In [ ]:
import functools
def score_match(m):
    factors = [int(m.group(v)) for v in ['x','y']]
    return factors[0] * factors[1]
score_match(m)

1476

In [ ]:
import operator, functools

def score(s:str) -> int:
    matches = re.finditer(pat,s)
    scores = (score_match(m) for m in matches)
    return sum(scores)

In [ ]:
score(sample)

161

In [ ]:
score(inp)

180233229

## prompt 2

There are two new instructions you'll need to handle:

The do() instruction enables future mul instructions.
The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

```
xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))
```

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (`2*4` + `8*5`).

In [ ]:
sample2 = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"""

In [ ]:
print(sample2)

xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))


In [ ]:
pat

'mul\\((?P<x>[0-9]{1,3}),(?P<y>[0-9]{1,3})\\)'

In [ ]:
import re
pat = r'(?P<mul>mul\((?P<x>[0-9]{1,3}),(?P<y>[0-9]{1,3})\))'
pat_do = r'(?P<do>do\(\))'
pat_dont = r"(?P<dont>don't\(\))"

In [ ]:
print(pat_dont)

(?P<dont>don't\(\))


In [ ]:
bigpat = f"{pat}|{pat_do}|{pat_dont}"

In [ ]:
m = re.match(bigpat,"do()")
m

<re.Match object; span=(0, 4), match='do()'>

In [ ]:
m.group('do')

'do()'

In [ ]:
m.group('dont')

In [ ]:
m = re.match(bigpat,"mul(123,45)")
m.group('mul')

'mul(123,45)'

In [ ]:
m.group('x')

'123'

In [ ]:
def score_matches(matches):
    total = 0
    enabled = True
    for m in matches:
        if m.group('dont'):
            enabled = False
        elif m.group('do'):
            enabled = True
        elif m.group('mul') and enabled:
            score = score_match(m)
            total += score
    return total

In [ ]:
score_matches(sample2)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[1], line 1
----> 1 score_matches(sample2)

Cell In[1], line 5, in score_matches(matches)
      3 enabled = True
      4 for m in matches:
----> 5     if m.group('dont'):
      6         enabled = False
      7     elif m.group('do'):

AttributeError: 'str' object has no attribute 'group'


AttributeError: 'str' object has no attribute 'group'

In [ ]:
matches = re.finditer(bigpat,sample2)
score_matches(matches)

48

In [ ]:
def score_corrupted(s:str) -> int:
    matches = re.finditer(bigpat,s)
    return score_matches(matches)

In [ ]:
score_corrupted(sample2)

48

In [ ]:
score_corrupted(inp)

95411583